In [163]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, plot_confusion_matrix
from sklearn import tree

In [164]:
pd.set_option("display.max_columns", None)

In [165]:
pump_target = pd.read_csv('data/Training_set_labels.csv')
pump_target.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [166]:
pump_features = pd.read_csv('data/Training_set_values.csv')
pump_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [167]:
pumps = pump_features.merge(pump_target)
pumps

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [6]:
# Droping columns about when pump data was recored due to not being useful.
pumps.drop(['date_recorded', 'recorded_by'], axis=1, inplace=True)

In [7]:
# Checking for na's.
pumps.isna().sum()

id                           0
amount_tsh                   0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity                     0
quantity

In [8]:
# Droping scheme_name first cuz will lose almost almost half data if drop all na's first.
pumps.drop('scheme_name', axis=1, inplace=True)
pumps.dropna(inplace=True)
pumps.isna().sum()

id                       0
amount_tsh               0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
s

In [9]:
# A few of the columns have only true or false values but are object dtypes.
# Changing this columns to bool dtypes.
pumps['public_meeting'] = pumps['public_meeting'].astype(bool)
pumps['permit'] = pumps['permit'].astype(bool)

In [10]:
pumps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48288 entries, 0 to 59399
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     48288 non-null  int64  
 1   amount_tsh             48288 non-null  float64
 2   funder                 48288 non-null  object 
 3   gps_height             48288 non-null  int64  
 4   installer              48288 non-null  object 
 5   longitude              48288 non-null  float64
 6   latitude               48288 non-null  float64
 7   wpt_name               48288 non-null  object 
 8   num_private            48288 non-null  int64  
 9   basin                  48288 non-null  object 
 10  subvillage             48288 non-null  object 
 11  region                 48288 non-null  object 
 12  region_code            48288 non-null  int64  
 13  district_code          48288 non-null  int64  
 14  lga                    48288 non-null  object 
 15  wa

In [11]:
# Looking at values for all object dtypes.
object_types = list(pumps.select_dtypes(object))
for column in object_types:
    print('-----------------')
    print(pumps[column].value_counts())

-----------------
Government Of Tanzania            8080
Danida                            2920
Hesawa                            1388
Kkkt                              1260
World Bank                        1086
                                  ... 
Tanzania Egypt Technical Co Op       1
Dominiki Simwen                      1
Tasaf/tlc                            1
Africaone Ltd                        1
Quick Wins Scheme                    1
Name: funder, Length: 1586, dtype: int64
-----------------
DWE                14637
Government          1438
RWE                 1031
Commu                893
KKKT                 885
                   ...  
CRISTAN OUTRICH        1
Friend from UN         1
Kwamdulu estate        1
Sua                    1
Water Aid/DWE          1
Name: installer, Length: 1787, dtype: int64
-----------------
none                    3356
Shuleni                 1354
Zahanati                 619
Msikitini                447
Bombani                  252
            

In [12]:
# This is a list of all object dtypes that > 100. 
over_100 = ['basin', 'region', 'extraction_type_class', 'payment', 'payment_type', 'quality_group', 'quantity', 
            'quantity_group', 'source_type', 'source_class']

# This is a list of all object dtypes that < 100.
under_100 = ['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_management', 'extraction_type', 
             'extraction_type_group', 'management', 'management_group', 'water_quality',  'source', 'waterpoint_type', 
             'waterpoint_type_group']

In [27]:
pumps.drop(under_100, axis=1, inplace=True)

In [28]:
pumps

,id,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,region_code,district_code,population,public_meeting,permit,construction_year,extraction_type_class,payment,payment_type,quality_group,quantity,quantity_group,source_type,source_class,status_group
0,69572,6000.0,1390,34.938093,-9.856322,0,Lake Nyasa,Iringa,11,5,109,True,False,1999,gravity,pay annually,annually,good,enough,enough,spring,groundwater,functional
2,34310,25.0,686,37.460664,-3.821329,0,Pangani,Manyara,21,4,250,True,True,2009,gravity,pay per bucket,per bucket,good,enough,enough,dam,surface,functional
3,67743,0.0,263,38.486161,-11.155298,0,Ruvuma / Southern Coast,Mtwara,90,63,58,True,True,1986,submersible,never pay,never pay,good,dry,dry,borehole,groundwater,non functional
5,9944,20.0,0,39.172796,-4.765587,0,Pangani,Tanga,4,8,1,True,True,2009,submersible,pay per bucket,per bucket,salty,enough,enough,other,unknown,functional
6,19816,0.0,0,33.362410,-3.766365,0,Internal,Shinyanga,17,3,0,True,True,0,handpump,never pay,never pay,good,enough,enough,borehole,groundwater,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394,11164,500.0,351,37.634053,-6.124830,0,Wami / Ruvu,Morogoro,5,6,89,True,True,2007,submersible,pay monthly,monthly,good,enough,enough,borehole,groundwater,non functional
59395,60739,10.0,1210,37.169807,-3.253847,0,Pangani,Kilimanjaro,3,5,125,True,True,1999,gravity,pay per bucket,per bucket,good,enough,enough,spring,groundwater,functional
59396,27263,4700.0,1212,35.249991,-9.070629,0,Rufiji,Iringa,11,4,56,True,True,1996,gravity,pay annually,annually,good,enough,enough,river/lake,surface,functional
59398,31282,0.0,0,35.861315,-6.378573,0,Rufiji,Dodoma,1,4,0,True,True,0,handpump,never pay,never pay,good,insufficient,insufficient,shallow well,groundwater,functional


In [42]:
# Looking at the numeric columns.
int64_types = list(pumps.select_dtypes('int64'))
float64_types = list(pumps.select_dtypes('float64'))
int_types = int64_types + float64_types
for column in int_types:
    print('-----------------')
    print(pumps[column].value_counts())

-----------------
2047     1
48477    1
28007    1
25958    1
32101    1
        ..
72310    1
68212    1
12915    1
14962    1
4098     1
Name: id, Length: 48288, dtype: int64
-----------------
 0       15783
-15         50
 1290       47
-13         43
 1295       43
         ...  
 2467        1
 2202        1
 2286        1
 2330        1
 2420        1
Name: gps_height, Length: 2426, dtype: int64
-----------------
0       47567
6          73
1          68
8          46
5          44
32         40
45         36
15         35
39         30
93         28
3          26
7          24
65         22
47         21
102        20
4          19
2          18
17         16
80         15
20         14
25         12
41         10
11         10
34         10
16          8
120         7
150         6
22          6
24          5
50          5
12          5
9           4
38          4
58          4
14          3
10          3
27          2
26          2
35          1
131         1
698         1
140

In [43]:
# List of numeric columns that have way to many 0's.
filled_0_value = ['gps_height', 'num_private', 'population', 'construction_year', 'amount_tsh', 'longitude']

# This numeric columns has 2.000000e-08 filled in the same amount of times that longitude has 0.
filled_2_value = ['latitude']

In [52]:
X = pumps.drop(['id', 'status_group'], axis=1)
y = pumps['status_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=pumps['status_group'])

In [49]:
# Numeric fearures from the data set.
numeric_features = ['region_code', 'district_code']

numeric_features_0 = filled_0_value

numeric_features_2 = filled_2_value

# Boolean fearures from the data set.
boolean_features = ['public_meeting', 'permit']

# Categorical fearures from the data set.
categorical_features = over_100

In [82]:
# Setting up the numeric, boolean, and cartegorical transformers.
numeric_transformer_0 = Pipeline(steps=[('imputer_0', SimpleImputer(missing_values=0)), 
                                        ('scaler', StandardScaler())])
                                          
numeric_transformer_2 = Pipeline(steps=[('imputer_2', SimpleImputer(missing_values=-2.000000e-08)),
                                        ('scaler', StandardScaler())])
                                          
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
                                          
boolean_transformer = Pipeline(steps=[('imputer_2', SimpleImputer(missing_values=0))])
    
categorical_transformer = Pipeline(steps=[('encoder', OrdinalEncoder())])

# Using ColumnTransformer to trasformer the columns to avoid data leakage.
preprocessor = ColumnTransformer(transformers=[('numeric_0', numeric_transformer_0, numeric_features_0),
                                               ('numeric_2', numeric_transformer_2, numeric_features_2),
                                               ('numeric', numeric_transformer, numeric_features),
                                               ('boolean', boolean_transformer, boolean_features),
                                               ('categorical', categorical_transformer, categorical_features)])

In [91]:
features = numeric_features_0 + numeric_features_2 + numeric_features + boolean_features + categorical_features

In [83]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [85]:
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train_transformed, y_train)

DecisionTreeClassifier(random_state=42)

In [86]:
y_hat = tree_model.predict(X_test_transformed)

In [87]:
recall_score(y_test, y_hat, average='micro')

0.7503313452617627

In [161]:
feature_importances = list(tree_model.feature_importances_)
feature_importances = [value*100 for value in feature_importances]
feature_importances = dict(zip(features, feature_importances))
sorted_dict = dict( sorted(feature_importances.items(),
                           key=lambda item: item[1],
                           reverse=True))
bottom_features = list(sorted_dict.keys())
bottom_features = bottom_features[-11:]
drop_features = bottom_features + ['id', 'status_group']

In [162]:
X = pumps.drop(drop_features, axis=1)
y = pumps['status_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=pumps['status_group'])